In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#! SETUP 1
import sys, os
_snlp_book_dir = "../../../../"
sys.path.append(_snlp_book_dir) 
import math
from glob import glob
from os.path import isfile, join
from statnlpbook.vocab import Vocab
from statnlpbook.scienceie import calculateMeasures
import shutil
import string

In [3]:
### General modules
import pprint
from gensim.models import fasttext
from gensim.models import KeyedVectors
from copy import deepcopy
from collections import Counter
import numpy as np
import pandas as pd

### Our own modules
from Extra_files.modules.load_and_save import load_scienceie, save_to_ann, reformat_to_save
from Extra_files.modules.rules import _add_rules
from Extra_files.modules.error_analysis import plot_confusion_matrix, get_dataframe
from Extra_files.modules.DataPreparation import entityLocator, addPOStoDic, addLemmatoDic, inputPair
from Extra_files.modules.MasterVocab import MasterVocab
from Extra_files.modules.scaling import downscale
from Extra_files.models.dummy_model import _sebastians_dummy_model
from Extra_files.modules.Outputter import checkLegality, mostProbableYetLegal
from Extra_files.modules.CalcMeasuresTMP import calcMeasures, normaliseAnn

pp = pprint.PrettyPrinter(indent=1,depth = None, compact = True).pprint

In [4]:
from pickle import load, dump

# data_m_XY = dataX_Y_format(train_data,indices = True) 
# data_m_XY_dev = dataX_Y_format(dev_data,indices = True)
# dump(data_m_XY, open('Extra_files/resources/train_ind.pkl', 'wb'))

data_m_XY = load(open('Extra_files/resources/train_ind.pkl', 'rb'))
data_m_XY_dev = load(open('Extra_files/resources/dev_ind.pkl', 'rb'))
dev_data = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "dev"))
if False:
    print(data_m_XY['metadata'][0])
    pp(data_m_XY['data_X'][0])
    print(data_m_XY['data_Y'][0])
    
data = dev_data[list(dev_data.keys())[10]] # this one has many relations

Number of entities removed due to overlap: 269 out of 1330
Number of entities not identified in text: 1 out of 1061
Number of relations lost due to overlap: 10 out of 168
Removed references.
Concatenated 'i.e.' and 'e.g.'.


In [5]:
pp(data['relations'])

[('*', 'Synonym', 'T4', 'T3'), ('*', 'Synonym', 'T6', 'T5'),
 ('R1', 'Hyponym', 'T15', 'T31'), ('R2', 'Hyponym', 'T16', 'T31'),
 ('R3', 'Hyponym', 'T17', 'T31'), ('R4', 'Hyponym', 'T18', 'T31'),
 ('R5', 'Hyponym', 'T19', 'T31')]


In [6]:
if False:
    predictions = np.array([[0.4, 0.7, 0.3, 0.1], [0.8, 0.2, 0.5, 0.1], [0.5, 0.3, 0.6, 0.2], [0.9, 0.1, 0.0, 0.0]])
    #mostProbableYetLegal(predictions)

    # rank by highest probability
    probs = np.max(predictions, axis = 1)
    highestorder = np.argsort(probs)[::-1] #flip the order
    print(highestorder)
    # sort predictions
    pred_sorted = predictions[highestorder]
    legalpreds = []
    checkpreds = []
    for pred in pred_sorted:
        print(legalpreds)
        checkpreds = deepcopy(legalpreds)
        checkpreds.append(pred)
        print(legalpreds)
        # append if legal
        if checkLegality(checkpreds):
            legalpreds.append(pred)
        #end-if
    #end-for
    print(legalpreds)

    print("the function")
    mostProbableYetLegal(predictions)

In [7]:
if False:
    targetann = open("fakeANN/fakePred/example.ann", "r")
    pp(normaliseAnn(targetann, ""))
    targetann.close()

In [17]:
calcMeasures("fakeANN/fakeGold/", "fakeANN/fakePred/")

1.ann file missing in fakeANN/fakePred/. Assuming no predictions are available for this file.
10.ann file missing in fakeANN/fakePred/. Assuming no predictions are available for this file.
11.ann file missing in fakeANN/fakePred/. Assuming no predictions are available for this file.


UnboundLocalError: local variable 'ent2' referenced before assignment

**What works and when:**<br>
[ E ] indicates error, [ C ] indicates that the case runs <br>
1.ann : [ C ] Number of entities is below 2<br>
2.ann : [ E ] Number of entities is below 2 and includes relations: **"local variable 'ent2' referenced before assignment"**<br>
3.ann : [ E ] Number of entities is below 2 and relations reference only non-existing entities: **"local variable 'ent1' referenced before assignment"**<br>
4.ann : [ E ] Relations reference non-existing entities:**"local variable 'ent1' referenced before assignment"**<br>
5.ann : [ C ] Relation between the same entity: Hyponym-of Arg1:T1 Arg2:T1<br>
6.ann : [ C ] Relation between the same entity: Synonym-of T1 T1<br>
7.ann : [ C ] No ANN file/misnamed (shouldn't happen but sure)<br>
8.ann : [ C ] Circular relations: Hyponym-of Arg1:T5 Arg2:T6 & Hyponym-of Arg1:T6 Arg2:T5 <br>
9.ann : [ C ] Conflicting relation on same entities: Synonym-of and Hyponym-of T1 T2 <br>
10.ann: [ C ] Duplicate relations <br>
11.ann: [ E ] Relations reference non-existing entities, BUT has some valid relations in the beginning <br>
12.ann: [ E ] Relations reference non-existing entities, BUT has only valid relations in the end (not in the beginning) <br>